<a href="https://colab.research.google.com/github/Trantracy/Spam-Classification/blob/master/Spam_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://developers.google.com/machine-learning/guides/text-classification/images/TextClassificationExample.png)

### “To spam, or not to spam, that is the question”

Before you start, please keep in mind the flow of our project today:  


1.   Get data
2.   Clean data
3.   Transform text data to numbers so that computer can understand (**vectorization**)
3.   Build our model using sklearn 
4.   Tune our model for higher accuracy 



## 1. Get the data

In [0]:
# we import our libraries

import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# hello world 

In [0]:
data_url = 'https://raw.githubusercontent.com/accidentallydoesnotwork/Spam_message/master/spam.csv'
data = pd.read_csv(data_url,  delimiter=',',encoding='latin-1')

In [0]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [0]:
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)

In [0]:
data.rename(columns={'v1':'label', 
                     'v2':'text'}, inplace=True)

In [0]:
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [0]:
data.shape

(5572, 2)

In [0]:
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## 2. Clean our Data

In [0]:
import nltk 
nltk.download('stopwords')
from nltk import stem 
from nltk.corpus import stopwords 
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

#### Stopwords




remove all the words that do not contain information

Quick question: Name me 10 stopwords?

In [0]:
# get stopwords
stopwords = set(stopwords.words('english'))

In [0]:
# remove all the stopwords function
def remove_stopwords(text):
    clean_text=[word for word in text if word not in stopwords]
    return clean_text 

In [0]:
def demon_stopwords(text):
    clean_text=' '.join(word for word in text if word not in stopwords)
    return clean_text

In [0]:
text = "What is love? Baby don't hurt me. don't hurt me No more. Baby don't hurt me, don't hurt me No more. What is love?"
demon_stopwords(text.split())

'What love? Baby hurt me. hurt No more. Baby hurt me, hurt No more. What love?'

### Text normalization 

<img src="https://i.imgur.com/0zNf5Ln.png" width="600">

---



#### Stemmer

Cut the end of the words to remove the surfixes (hopefully!)

In [0]:
# stemmer = stem.SnowballStemmer('english')
stemmer = stem.PorterStemmer()

In [0]:
def stemming(text): 
    stem = [stemmer.stem(word) for word in text]
    return stem

In [0]:
test = 'rock rocks corpora corpus foot feet beautiful beauty available availability loved loving love. loving. '
print("Clean with stem:", stemming(test.split()), sep='\n')

Clean with stem:
['rock', 'rock', 'corpora', 'corpu', 'foot', 'feet', 'beauti', 'beauti', 'avail', 'avail', 'love', 'love', 'love.', 'loving.']


**What we learn here:** punctuation affects the ability to clean data. So we need to remove the punctuation

In [0]:
import string
import re
# string.punctuation is just a list of special characters 
string.punctuation


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [0]:
# remove all the special characters above 
def remove_punctuation(text):
    new_text=''.join([char for char in text if char not in string.punctuation])
    return new_text

In [0]:
# split the sentence into a list of words
def tokenize(text):
    tokens=re.split('\W+',text.lower())
    return tokens 

In [0]:
# Visualize our cleaning process 
new_data = data.copy()
new_data['punctuation_free'] = new_data['text'].apply(remove_punctuation)
new_data['tokenize'] = new_data['text'].apply(tokenize)
new_data['remove_stopwords'] = new_data['tokenize'].apply(remove_stopwords)
new_data['stemming'] = new_data['remove_stopwords'].apply(stemming)

In [0]:
new_data.head()

,label,text,punctuation_free,tokenize,remove_stopwords,stemming
0,ham,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n...","[go, jurong, point, crazi, avail, bugi, n, gre..."
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni, ]","[ok, lar, joking, wif, u, oni, ]","[ok, lar, joke, wif, u, oni, ]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entri, 2, wkli, comp, win, fa, cup, fin..."
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say, ]","[u, dun, say, earli, hor, u, c, alreadi, say, ]"
4,ham,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...,"[nah, i, don, t, think, he, goes, to, usf, he,...","[nah, think, goes, usf, lives, around, though]","[nah, think, goe, usf, live, around, though]"


**Now that we understand the concept of each cleanning method, let's apply it to our data**

In [0]:
# Clean our data using stopwords and stemming 

def clean_message_stem(msg):
  # all lower case 
  msg = msg.lower()
  # removing stopwords
  msg = [word for word in msg.split() if word not in stopwords]
  # using stemmer
  msg = " ".join([stemmer.stem(word) for word in msg])
  
  return msg 

In [0]:
# Clean our data using stopwords and lemmatization 

def clean_message_lemma(msg):
  # all lower case 
  msg = msg.lower()
  # removing stopwords
  msg = [word for word in msg.split() if word not in stopwords]
  # using lemma
#   print(word for word in msg)
  msg = " ".join([lemmatizer.lemmatize(word) for word in msg])
  
  return msg 

In [0]:
# remove special characters and clean message using stemming.

data['text'] = data['text'].apply(remove_punctuation)
data['text'] = data['text'].apply(clean_message_stem)

## 3. Transform text data to numbers so that computer can understand

In [0]:
# prep our data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size = 0.1, random_state = 1)

![Visualize train test split](https://www.researchgate.net/profile/Brian_Mwandau/publication/325870973/figure/fig6/AS:639531594285060@1529487622235/Train-Test-Data-Split.png)

**Visualize train test split.**

In [0]:
X_test.head()

1078                                        convey regard
4028                           û anyway mani good even u
958     sort code acc bank natwest repli confirm ive s...
4642                                sorri din lock keypad
4674    hi babe chloe r u smash saturday night great w...
Name: text, dtype: object

### Bag of word

Bag of word (BOW) counts the frequency of each word in the documents. Then a text will be represented as the bag of its word (each word frequency), disregarding both grammar and word order. 

![alt text](https://cdn-media-1.freecodecamp.org/images/1*j3HUg18QwjDJTJwW9ja5-Q.png)

In [0]:
# we import the CountVectorizer, which is the bag of word method
from sklearn.feature_extraction.text import CountVectorizer

# assign it to a variable 
count_vectorizer = CountVectorizer()

# transform our sentence into a vector of number 
X_train_bag = count_vectorizer.fit_transform(X_train)

In [0]:
# visualize our data after transform 
BoW_data = pd.DataFrame(X_train_bag.toarray())
BoW_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,7583,7584,7585,7586,7587,7588,7589,7590,7591,7592,7593,7594,7595,7596,7597,7598,7599,7600,7601,7602,7603,7604,7605,7606,7607,7608,7609,7610,7611,7612,7613,7614,7615,7616,7617,7618,7619,7620,7621,7622
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### TfIDF

TFIDF (Term Frequency-Inverse Document Frequency):  
This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus

**Example: Consider a document containing 100 words wherein the word cat appears 3 times. The term frequency (i.e., tf) for cat is then (3 / 100) = 0.03. Now, assume we have 10 million documents and the word cat appears in one thousand of these. Then, the inverse document frequency (i.e., idf) is calculated as log(10,000,000 / 1,000) = 4. Thus, the Tf-idf weight is the product of these quantities: 0.03 * 4 = 0.12.**

In [0]:
# Import tfidf method 
from sklearn.feature_extraction.text import TfidfVectorizer

# assign the tfidf method to a variable 
vectorizer = TfidfVectorizer()

# transform our sentences into vectors of numbers using tfidf method 
X_train_tfidf = vectorizer.fit_transform(X_train)

In [0]:
# visualize our vectors
tfidf_data = pd.DataFrame(X_train_tfidf.toarray())
tfidf_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,7583,7584,7585,7586,7587,7588,7589,7590,7591,7592,7593,7594,7595,7596,7597,7598,7599,7600,7601,7602,7603,7604,7605,7606,7607,7608,7609,7610,7611,7612,7613,7614,7615,7616,7617,7618,7619,7620,7621,7622
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Now we have finish both cleaning our data and transform it into numbers that machine can understand. The only thing left is to build the classifier to separate spam emails**


![](https://i.imgur.com/A12EMUd.png)

## 4. Build our model using sklearn

### RandomForest



<img src="https://matthewdharriscom.files.wordpress.com/2016/07/dog_2layer.png?w=350&h=300" width="400">

**A simple decision tree**

TF-IDF performance

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report


In [0]:
rfc = RandomForestClassifier(n_estimators=20, random_state=0)
rfc.fit(X_train_tfidf, y_train)
X_test_tfidf = vectorizer.transform(X_test)
y_pred = rfc.predict(X_test_tfidf)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[490   0]
 [ 11  57]]
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       490
        spam       1.00      0.84      0.91        68

    accuracy                           0.98       558
   macro avg       0.99      0.92      0.95       558
weighted avg       0.98      0.98      0.98       558



Bag of word performance 

In [0]:
rfc = RandomForestClassifier(n_estimators=20, random_state=0)
rfc.fit(X_train_bag, y_train)
X_test_bow = count_vectorizer.transform(X_test)
y_pred = rfc.predict(X_test_bow)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[489   1]
 [ 11  57]]
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       490
        spam       0.98      0.84      0.90        68

    accuracy                           0.98       558
   macro avg       0.98      0.92      0.95       558
weighted avg       0.98      0.98      0.98       558



### K-nearest neighbors

![alt text](http://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1531424125/KNN_final_a1mrv9.png)

**TF-IDF performance**

In [0]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=2, weights='distance')
knn.fit(X_train_tfidf, y_train)
y_pred = knn.predict(X_test_tfidf)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[490   0]
 [ 16  52]]
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98       490
        spam       1.00      0.76      0.87        68

    accuracy                           0.97       558
   macro avg       0.98      0.88      0.93       558
weighted avg       0.97      0.97      0.97       558



**Bag of word performance**

In [0]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=2, weights='distance')
knn.fit(X_train_bag, y_train)
y_pred = knn.predict(X_test_bow)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[490   0]
 [ 16  52]]
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98       490
        spam       1.00      0.76      0.87        68

    accuracy                           0.97       558
   macro avg       0.98      0.88      0.93       558
weighted avg       0.97      0.97      0.97       558



What you can learn from the building model session: 


1.   You can fine tune your model by adjusting its hyperparameters. So it is extremely crucial to understand how the models works. Sklearn is easy to use, but you need to understand the model
2.   You can have better result just by making your input better. Simple model can outperform complex model with better input. **"Garbage in, Garbage out"**, please remember that



![alt text](https://static.packt-cdn.com/products/9781788838535/graphics/e6a200ba-94b2-4c9f-a9af-5e5812cfa8a6.jpg)

**Congratulations, you have finished building your first email classification model. Now you can test and have fun with it**

In [0]:
# a function to 
def pred(msg):
    msg = vectorizer.transform([msg])
    prediction = knn.predict(msg) # you can change between knn and rfc
    return prediction[0]

In [0]:
pred('hey baby, wanna netflix and chill tonight, please call me 0123123. Please click yes')

'ham'

In [0]:
pred("Join our workshop now. Available only in limited time")

'ham'

In [0]:
pred("You have 1 new voicemail. Please call 08719181503")

'ham'

In [0]:
pred(spam)

NameError: ignored

In [0]:
spam = data[data['label']=='spam'].sample(10)['text'].values[9]

## 5. Improve your accuracy

Your model has been finished, but you discovered that despite the high accuracy, your model still fails to recognize spam messages. Therefore your next task is to try and improve your model. Here are some problems that your model may has and way to fix them: 

1. Poor prediction due to lack of data.

  **Solution**: ***Get more data***, or try to tune the parameter
2. Data imbalance: one class has more data than the others. 

  **Solution**: ***Get more data***, or use undersampling, oversampling technique 
3. Poor input: either due to vectorization or unrealistic input. 

  **Solution**: ***Get more data***, clean data more carefully, or try different technique to manipulate the input (Hint: Word2Vect)
4. Your current model is not strong enough. 

  **Solution**: ***Get more data***, or tune the parameter, or switch to a different model. 

## Bag of word example 

In [0]:
import nltk 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
import collections, re

In [0]:
bow = CountVectorizer()
text = 'I love this dog'
text1 = 'What is love? Baby dont hurt me, dont hurt me, no more'
text2 = 'We loving this dog so much'
text3 = 'I loved that toy dog. Too bad it broke'

In [0]:
dictionary = [text, text1, text2, text3]
dictionary

In [0]:
lst = ' '.join(dictionary).split()

In [0]:
bagofwords = collections.Counter(' '.join(dictionary).split())
bagofwords
# bagofwords.get('dog')

In [0]:
def vectorize(text, bagofword):
  vector = []
  for word in text.split():
    if word in bagofword.keys():
      vector.append(bagofword.get(word))
    else:
      vector.append(0)
  print(text, vector)


In [0]:
test = 'Dog is man best friend. We should not eat dog'
vectorize(test, bagofwords)

In [0]:
nltk.download('stopwords')
from nltk.corpus import stopwords

In [0]:
stopword = set(stopwords.words('english'))


In [0]:
def clean_stopwords(dictionary):
  dictionary = [word for word in dictionary if word not in stopword]
  bow = collections.Counter(dictionary)
  return bow

In [0]:
bow = clean_stopwords(lst)
bow

In [0]:
vectorize(test, bow)

In [0]:
import string 
special_char = string.punctuation + '?'
special_char

In [0]:
def remove_punctuation(text):
    new_text=''.join([char for char in text if char not in string.punctuation])
    return new_text


In [0]:
for i in lst: 
  lst.append(remove_punctuation(i))